In [3]:
from telegram import Bot
import logging
import requests
from datetime import datetime
from dateutil import tz
import pandas as pd
import re
import locale

locale.setlocale(locale.LC_TIME, 'ru_RU.UTF-8')

class CalendarAPI:
    def __init__(self, base_url, params={}, cookies={}, headers={}):
        self.base_url = base_url
        self.cookies = cookies
        self.headers = headers
        self.params = params

    def fetch_events(self, year, month):
        json_data = [{
            'index': 0,
            'methodname': 'core_calendar_get_calendar_monthly_view',
            'args': {
                'year': str(year),
                'month': str(month),
                'courseid': 1,
                'day': 1,
                'view': 'month',
            },
        }]

        response = requests.post(self.base_url, params=self.params,
                                 cookies=self.cookies, headers=self.headers, json=json_data)
        return response.json()[0].get('data', {})


class EventParser:
    @staticmethod
    def get_date(input_string):
        pattern = r'(\d{2}:\d{2})'
        match = re.search(pattern, input_string)
        return match.group(1) if match else "No match found"

    @staticmethod
    def date_formatter(date_str, lang='ru'):
        if lang == 'ru':
            res = [val.capitalize()
                   for val in date_str.replace('События - ', '').split()]
            return f"{res[0]}, {' '.join(res[1:])}"
        elif lang == 'en':
            return ' '.join([val.strip().capitalize() for val in date_str.replace('events', '').strip().split()])

    @staticmethod
    def flatten(lst):
        return [item for sublist in lst for item in sublist]


class EventDataProcessor:
    def __init__(self, data):
        self.data = data

    def extract_events(self):
        events = [[day for day in week.get('days', []) if day.get(
            'hasevents')] for week in self.data.get('weeks', [])]
        events_flat = EventParser.flatten(events)

        result = []
        for event in events_flat:
            for e in event.get('events', [{}]):
                result.append([
                    event.get('viewdaylink', ''),
                    str(datetime.fromtimestamp(
                        event.get('timestamp'), tz=tz.gettz("GMT+3"))),
                    event.get('viewdaylinktitle', ''),
                    e.get('activityname', ''),
                    e.get('viewurl'),
                    e.get('url', ''),
                    e.get('description', ''),
                    e.get('formattedtime'),
                    e.get('course', {}).get('fullnamedisplay'),
                    e.get('course', {}).get('viewurl', '')
                ])
        return result


class CalendarOutput:
    def __init__(self, events):
        self.events = events
        self.df = pd.DataFrame(events)
        self.df.columns = ['calendar_link', 'date', 'date_str', 'name', 'calendar',
                           'hw_link', 'description', 'due_time', 'subj', 'subj_link']

    @staticmethod
    def to_bold(string):
        return f'*{string}*'

    @staticmethod
    def to_clickable_link(link_text, link):
        return f'[{link_text}]({link})'

    @staticmethod
    def get_russian_now() -> str:    
        now = datetime.now()
        return datetime.strftime(now, '%A, %d %B %Y %H:%M')

    def generate_output(self):
        output = '📌 ' + \
            f'{self.to_bold("Время последнего обновления:")} {self.get_russian_now()} \n'
        self.df = self.df.sort_values('date').reset_index(drop=True)
        for dt in self.df.date_str.unique():
            output += '\n' + '📅 ' + EventParser.date_formatter(dt) + '\n'
            cur_df = self.df[self.df.date_str == dt]
            for idx in range(len(cur_df)):
                # output += self.to_clickable_link(cur_df.iloc[idx].subj, cur_df.iloc[idx].subj_link) + "\n"
                output += cur_df.iloc[idx].subj + "\n"
                output += self.to_clickable_link(
                    cur_df.iloc[idx]['name'], cur_df.iloc[idx].hw_link) + "\n"
                output += self.to_bold(
                    f'До {EventParser.get_date(cur_df.iloc[idx].due_time)}') + '❗️' + '\n'

        return output


if __name__ == "__main__":
    cookies = {
        'COOKIES':''
    }

    headers = {
        'HEADERS':''
    }

    params = {
        'sesskey': 'SESS_KEY',
        'info': 'core_calendar_get_calendar_monthly_view',
    }

    # Set up logging
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)

    # Replace 'YOUR_BOT_TOKEN' with your bot's API token
    TOKEN = 'TOKEN'
    # Replace '@your_channel_username' with your channel's username (e.g., '@my_channel')
    CHANNEL_ID = 'CHANNEL_ID'

    api = CalendarAPI(base_url='https://edu.hse.ru/lib/ajax/service.php',
                      params=params, cookies=cookies, headers=headers)
    logger.info("Successfully connected to the API!")
    events = []
    for month in (9, 10):
        logger.info(f"Currently processeed month number {month}!")
        data = api.fetch_events(year=datetime.now().year, month=month)
        processor = EventDataProcessor(data)
        events.extend(processor.extract_events())
        logger.info(f"Got {len(events)} new events!")
    calendar_output = CalendarOutput(events)
    output = calendar_output.generate_output()
    logger.info("Successfully generated output")

    bot = Bot(token=TOKEN)

    # Send a message to the channel
    try:
        # Edit the message
        bot.edit_message_text(
            chat_id=CHANNEL_ID, parse_mode='markdown', message_id=1129, text=output)
        logger.info("Message edited successfully!")

    except Exception as e:
        logger.error(f"Failed to send or edit message: {e}")

📌 **Время последнего обновления:** 2024-09-11 22:31:30 

📅 Суббота, 14 Сентября
[Python для инженерии данных (2024/2025 модули: 1,2)](https://edu.hse.ru/course/view.php?id=211078)
[Домашнее задание 1](https://edu.hse.ru/mod/assign/view.php?id=1280150)
**До 23:59**❗️

📅 Понедельник, 16 Сентября
[Информационные системы (2024/2025 модули: 1,2)](https://edu.hse.ru/course/view.php?id=211074)
[Домашнее задание по теме 1 «Введение в информационные системы»](https://edu.hse.ru/mod/assign/view.php?id=1296563)
**До 16:00**❗️
[Семинар наставника (2024/2025 модули: 1,2,3,4)](https://edu.hse.ru/course/view.php?id=211075)
[Паспорт компетенций](https://edu.hse.ru/mod/assign/view.php?id=1290223)
**До 23:59**❗️

📅 Понедельник, 23 Сентября
[Информационные системы (2024/2025 модули: 1,2)](https://edu.hse.ru/course/view.php?id=211074)
[Домашнее задание по теме 2 «Методологии и инструменты создания информационных систем»](https://edu.hse.ru/mod/assign/view.php?id=1296574)
**До 16:00**❗️

📅 Суббота, 28 Сент

In [ ]:
# tg @nikitaworonov для доработок
# TODO Может развернуть и захостить все это в кроне
# TODO Может по файликам лучше в проект отдельные это дело раскидать